In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from zipfile import ZipFile

with ZipFile("/content/data (1).zip", 'r') as zObject:
    zObject.extractall(path="/content/")

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0:"SOS", 1:"EOS"}
    self.n_words = 2
  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)
  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [4]:
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
  )

def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r"([.!?])", r" \1", s)
  s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
  return s.strip()

In [5]:
def readLangs(lang1, lang2, reverse=False):
  print("Reading Lines......")

  lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

  pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

  if reverse:
    pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang2)
    output_lang = Lang(lang1)
  else:
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

  return input_lang, output_lang, pairs

In [6]:
MAX_LEN = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
  return len(p[0].split(' ')) < MAX_LEN and len(p[1].split(' ')) < MAX_LEN #and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
  return [p for p in pairs if filterPair(p)]

In [7]:
def prepareData(lang1, lang2, reverse = False):
  input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
  print("Read %s sentence pairs" %len(pairs))
  pairs = filterPairs(pairs)
  print("Trimmed to %s sentence pairs" % len(pairs))
  print("Counting words....")
  for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
  print("Counted Words")
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng','fra',True)

Reading Lines......
Read 135842 sentence pairs
Trimmed to 105692 sentence pairs
Counting words....
Counted Words
fra 17864
eng 10698


In [8]:
print(random.choice(pairs))

['si je le savais je vous le dirais', 'if i knew that i d tell you']


In [9]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size, dropout_p=0.1):
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, input):
    embedded = self.dropout(self.embedding(input))
    # print('embedded :', embedded.shape)
    output, hidden = self.gru(embedded)
    # print('output :', output.shape)
    # print('Hidden :', hidden[0].shape)
    return output, hidden

In [10]:
class BahdanauAttention(nn.Module):
  def __init__(self, hidden_size):
    super(BahdanauAttention,self).__init__()
    self.Wa = nn.Linear(hidden_size, hidden_size)
    self.Ua = nn.Linear(hidden_size, hidden_size)
    self.Va = nn.Linear(hidden_size, 1)

  def forward(self, query, keys):
    # print(self.Wa(query).shape)
    # print(self.Ua(keys).shape)
    scores = self.Va(torch.tanh( self.Wa(query) + self.Ua(keys) ))
    # print('Scores :', scores.shape)
    scores = scores.squeeze(2).unsqueeze(1)
    # print('Scores :', scores.shape)

    weights = F.softmax(scores, dim = -1)
    # print('Weights : ', weights.shape)
    context = torch.bmm(weights, keys)
    # print('context : ', context.shape)
    return context, weights

In [11]:
# att = BahdanauAttention(128)

In [12]:
# query = torch.randn((32, 1 ,128))
# keys = torch.randn((32, 10, 128))

In [13]:
# context, weights = att.forward(query, keys)

In [14]:
class AttnDecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size, dropout_p=0.1):
    super(AttnDecoderRNN, self).__init__()
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.attention = BahdanauAttention(hidden_size)
    self.gru = nn.GRU(2*hidden_size, hidden_size, batch_first=True)
    self.out = nn.Linear(hidden_size, output_size)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
    batch_size = encoder_outputs.size(0)
    decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
    # print('initial_decoder_inp : ', decoder_input.shape)
    decoder_hidden = encoder_hidden
    # print('decoder_hidden:', decoder_hidden.shape)
    decoder_outputs = []
    attentions = []
    for i in range(MAX_LEN):
      decoder_output, decoder_hidden, attn_weights = self.forward_step(
          decoder_input, decoder_hidden, encoder_outputs
      )
      decoder_outputs.append(decoder_output)
      attentions.append(attn_weights)

      if target_tensor is not None:
        decoder_input = target_tensor[:,i].unsqueeze(1)

      else:
        _, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze(-1).detach()

    decoder_outputs = torch.cat(decoder_outputs, dim=1)
    decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
    attentions = torch.cat(attentions, dim=1)

    return decoder_outputs, decoder_hidden, attentions
  def forward_step(self, inp, hidden, encoder_outputs):
    # print('inp : ', inp.shape)
    embedded = self.dropout(self.embedding(inp))
    # print('hidden : ', hidden.shape)
    query = hidden.permute(1,0,2)
    # print('query : ', query.shape)
    # print('enc_out : ', encoder_outputs.shape)
    context, attn_weights = self.attention(query, encoder_outputs)
    input_gru = torch.cat((embedded, context), dim=2)
    # print('input_gru :', input_gru.shape)
    output, hidden  = self.gru(input_gru, hidden)
    output = self.out(output)

    return output, hidden, attn_weights


In [15]:
# enc = EncoderRNN(input_lang.n_words, hidden_size=128)

In [16]:
# rand_inp = torch.randint(1, input_lang.n_words, size=(32,10))

In [17]:
# output, hidden = enc.forward(rand_inp)

In [18]:
# dec = AttnDecoderRNN(128, output_lang.n_words)

In [19]:
# ran_tar = torch.randint(1, output_lang.n_words, size=(32,10))

In [20]:
# out, hid, att = dec.forward(output, hidden, ran_tar)

In [21]:
def indexesFromSentence(lang, sentence):
  return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
  indexes = indexesFromSentence(lang, sentence)
  indexes.append(EOS_token)
  return torch.tensor(indexes, dtype=torch.long, device=device).view(1,-1)

def tensorsFromPair(pair):
  input_tensor = tensorFromSentence(input_lang, pair[0])
  target_tensor = tensorFromSentence(output_lang, pair[1])
  return (input_tensor, target_tensor)

def get_dataloader(batch_size):
  input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

  n = len(pairs)
  input_ids = np.zeros((n,MAX_LEN), dtype=np.int32)
  target_ids = np.zeros((n,MAX_LEN), dtype=np.int32)

  for idx, (inp, tgt) in enumerate(pairs):
    inp_ids = indexesFromSentence(input_lang, inp)
    tgt_ids = indexesFromSentence(output_lang, tgt)
    inp_ids.append(EOS_token)
    tgt_ids.append(EOS_token)
    input_ids[idx, :len(inp_ids)] = inp_ids
    target_ids[idx, :len(tgt_ids)] = tgt_ids

  train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                             torch.LongTensor(target_ids).to(device))

  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  return input_lang, output_lang, train_dataloader



In [22]:
def train_epoch(data_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):

  total_loss = 0
  for data in data_loader:
    input_tensor, target_tensor = data
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_outputs, encoder_hidden = encoder(input_tensor)
    decoder_outputs,_,_ = decoder(encoder_outputs, encoder_hidden, target_tensor)
    loss = criterion(
        decoder_outputs.view(-1, decoder_outputs.size(-1)),
        target_tensor.view(-1)
    )
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    total_loss += loss.item()
  return total_loss / len(data_loader)


In [23]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [24]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [25]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
  start = time.time()
  plot_losses = []
  print_loss_total = 0
  plot_loss_total = 0
  encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
  criterion = nn.NLLLoss()

  for epoch in range(1, n_epochs + 1):
    loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))


In [26]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)

    encoder_outputs, encoder_hidden = encoder(input_tensor)

    decoder_outputs, decoder_hidden, decoder_attention = decoder(encoder_outputs, encoder_hidden)

    _, topi = decoder_outputs.topk(1)

    decoded_ids = topi.squeeze()
    decoded_words = []

    for idx in decoded_ids:
      if idx.item()==EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(output_lang.index2word[idx.item()])
  return decoded_words, decoder_attention

In [27]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [28]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading Lines......
Read 135842 sentence pairs
Trimmed to 105692 sentence pairs
Counting words....
Counted Words
fra 17864
eng 10698
5m 38s (- 84m 41s) (5 6%) 1.3193
11m 13s (- 78m 33s) (10 12%) 0.6265
16m 47s (- 72m 45s) (15 18%) 0.4634
22m 20s (- 67m 1s) (20 25%) 0.3828
27m 55s (- 61m 25s) (25 31%) 0.3331
33m 30s (- 55m 50s) (30 37%) 0.2985
39m 8s (- 50m 19s) (35 43%) 0.2731
44m 44s (- 44m 44s) (40 50%) 0.2536
50m 22s (- 39m 10s) (45 56%) 0.2378
55m 56s (- 33m 34s) (50 62%) 0.2253
61m 31s (- 27m 57s) (55 68%) 0.2148
67m 6s (- 22m 22s) (60 75%) 0.2058
72m 39s (- 16m 46s) (65 81%) 0.1981
78m 12s (- 11m 10s) (70 87%) 0.1918
83m 45s (- 5m 35s) (75 93%) 0.1860
89m 19s (- 0m 0s) (80 100%) 0.1808


In [29]:
evaluateRandomly(encoder, decoder, 20)

> ce fut extremement douloureux
= it was excruciating
< it was excruciating <EOS>

> nous sommes impatients de vous voir
= we are looking forward to seeing you soon
< we can t wait to meet you d <EOS>

> tout le monde fut surpris
= everyone was surprised
< everyone was surprised surprised <EOS>

> vous n etes pas les seuls avec ce probleme
= you re not the only one with this problem
< you re not the only one with this problem <EOS>

> ou es tu exactement ?
= where exactly are you ?
< where exactly did you get ? <EOS>

> puis je partir maintenant ?
= can i go now ?
< can i go now ? <EOS>

> nous ne sommes pas en forme
= we re not fit
< we re not fit to make <EOS>

> he ouvrez la porte
= hey open the door
< hey open the door open the door <EOS>

> votre fille n est plus une enfant
= your daughter is not a child any more
< your daughter is more like a child any more <EOS>

> comment l as tu connu ?
= how did you get to know him ?
< how did you know him about it ? <EOS>

> pourquoi ne l as